In [5]:
import pandas_ta as ta
import pandas as pd
import numpy as np
from datetime import datetime
import main_functions
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
import warnings
# from lightweight_charts import Chart
# from lightweight_charts.widgets import JupyterChart
# import mplfinance as mpf
# import yfinance as yf
# Filter out the FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

In [6]:
import importlib
# If you want to update the module:
importlib.reload(main_functions)
from main_functions import *

In [ ]:
# exchange = krakenActive(mode)

In [7]:
# Initialize notebook mode
init_notebook_mode(connected=True)

In [8]:
pd.set_option('display.max_rows', 500)  # Set the maximum number of rows to display
pd.set_option('display.max_columns', 40) 

# Define Functions

In [9]:
def calculate_gann_signals(df, max_sw_cnt = 3, exit_perc = (80*0.01), side = "long"):
    # print(f'Exit percentage {side}: {exit_perc}')
    calculate_candle_type(df)
    
    # Initialize p_cnt with a list containing the initial value (0) for the first row
    p_cnt_values = [0]

    # Iterate over the rows of the DataFrame starting from the second row (index 1)
    for i in range(1, len(df)):
        # Get the current candlestick type for the current row
        current_candle_type = df['candle_type'].iloc[i]

        # Get the previous sw_cnt value (p_cnt) for the current row
        previous_sw_cnt = p_cnt_values[i - 1]

        # Initialize a variable to store the new sw_cnt value
        new_sw_cnt = None

        # Check various conditions to determine the new sw_cnt value
        if (current_candle_type == "up_bar" and previous_sw_cnt < 0) or (current_candle_type == "outside_dn_up" and previous_sw_cnt < 0):
            new_sw_cnt = 1
        elif current_candle_type == "up_bar" and previous_sw_cnt < max_sw_cnt:
            new_sw_cnt = previous_sw_cnt + 1
        elif current_candle_type == "up_bar" and previous_sw_cnt == max_sw_cnt:
            new_sw_cnt = max_sw_cnt
        elif (current_candle_type == "down_bar" and previous_sw_cnt > 0) or (current_candle_type == "outside_up_dn" and previous_sw_cnt > 0):
            new_sw_cnt = -1
        elif current_candle_type == "down_bar" and previous_sw_cnt > -max_sw_cnt:
            new_sw_cnt = previous_sw_cnt - 1
        elif current_candle_type == "down_bar" and previous_sw_cnt == -max_sw_cnt:
            new_sw_cnt = -max_sw_cnt
        elif current_candle_type == "outside_dn_up" and previous_sw_cnt > 0:
            new_sw_cnt = previous_sw_cnt
        elif current_candle_type == "outside_up_dn" and previous_sw_cnt < 0:
            new_sw_cnt = previous_sw_cnt
        else:
            # If none of the conditions are met, keep the sw_cnt unchanged
            new_sw_cnt = previous_sw_cnt

        # Append the new sw_cnt value to the list of sw_cnt values
        p_cnt_values.append(new_sw_cnt)

    # Check if the length of sw_cnt values matches the number of rows in df
    if len(p_cnt_values) == len(df):
        # Assign the list of sw_cnt values to a new column 'sw_cnt' in df
        df[f'sw_cnt_{side}'] = p_cnt_values
    else:
        print("Length mismatch error between p_cnt_values and df.")

    # Initialize sw_trend with NaN
    df[f'sw_trend_{side}'] = np.nan

    # Create mask1 and mask2
    mask1 = (
        ((df[f'sw_cnt_{side}'] == -(max_sw_cnt)) |
         ((df[f'sw_cnt_{side}'] == 1) & (df['candle_type'] == 'outside_dn_up'))) &
        (df[f'sw_cnt_{side}'].shift(1) == -(max_sw_cnt-1))  # & (df['sw_trend'].shift(1) != df['sw_trend'])
    )

    mask2 = (
        ((df[f'sw_cnt_{side}'] == (max_sw_cnt)) |
         ((df[f'sw_cnt_{side}'] == -1) & (df['candle_type'] == 'outside_up_dn'))) &
        (df[f'sw_cnt_{side}'].shift(1) == (max_sw_cnt-1))  # & (df['sw_trend'].shift(1) != df['sw_trend'])
    )

    # Update sw_trend based on mask1 and mask2
    df.loc[mask1, f'sw_trend_{side}'] = -1
    df.loc[mask2, f'sw_trend_{side}'] = 1

    # Forward fill the sw_trend column to carry forward the last value
    df[f'sw_trend_{side}'].ffill(inplace=True)

    trend_cnt = [0]

    # Loop through the DataFrame index
    for i in range(1, len(df)):
        if df[f'sw_trend_{side}'].iloc[i - 1] == df[f'sw_trend_{side}'].iloc[i]:
            trend_cnt.append(trend_cnt[i - 1] + 1)
        else:
            trend_cnt.append(1)
    # Append a 0 at the beginning to match the length of the DataFrame
    trend_cnt.insert(0, 0)

    df[f'trend_cnt_{side}'] = trend_cnt[:-1]

    # Initialize sw_top column with False
    df[f'sw_top_{side}'] = False
    df[f'sw_bottom_{side}'] = False

    # Calculate the maximum High value of past n candles when mask1 is true
    for i in range(len(df)):
        if mask1[i] & (df[f'sw_trend_{side}'].iloc[i - 1] != df[f'sw_trend_{side}'].iloc[i]):
            # Calculate the maximum High and its index
            high_range = df.loc[
                df.index[max(0, i - int(df[f'trend_cnt_{side}'][i]))]:df.index[i]]['High']
            max_high = high_range.max()
            max_high_index = high_range.idxmax()

            # Mark the row where max_high occurs as True
            df.at[max_high_index, f'sw_top_{side}'] = True

        elif mask2[i] & (df[f'sw_trend_{side}'].iloc[i - 1] != df[f'sw_trend_{side}'].iloc[i]):
            # Calculate the maximum High and its index
            low_range = df.loc[
                df.index[max(0, i - int(df[f'trend_cnt_{side}'][i]))]:df.index[i]]['Low']
            min_low = low_range.min()
            min_low_index = low_range.idxmin()

            # Mark the row where max_high occurs as True
            df.at[min_low_index, f'sw_bottom_{side}'] = True

    df[f'sw_high_price_{side}'] = np.where(
        df[f'sw_top_{side}'] == True, df['High'], np.nan)
    df[f'sw_high_price_{side}'].fillna(method='ffill', inplace=True)
    df[f'sw_low_price_{side}'] = np.where(
        df[f'sw_bottom_{side}'] == True, df['Low'], np.nan)
    df[f'sw_low_price_{side}'].fillna(method='ffill', inplace=True)

    # Filter rows where sw_top is True and calculate sw_highs
    df_tops = df[df[f'sw_top_{side}'] == True].copy()
    df_tops[f'sw_highs_{side}'] = np.where(
        df_tops['High'] > df_tops['High'].shift(1), "HH", "LH")

    # Filter rows where sw_bottom is True and calculate sw_lows
    df_bottoms = df[df[f'sw_bottom_{side}'] == True].copy()
    df_bottoms[f'sw_lows_{side}'] = np.where(
        df_bottoms['Low'] < df_bottoms['Low'].shift(1), "LL", "HL")

    # Concatenate the DataFrames and select the desired columns
    df_swings = pd.concat([df_tops, df_bottoms], axis=1)

    df_swings[f'trend_{side}'] = np.nan
    df_swings[f'trend_{side}'] = np.where(((df_swings[f'sw_highs_{side}'] == 'HH') & (df_swings[f'sw_lows_{side}'].shift(1) == 'HL')) |
                                  ((df_swings[f'sw_lows_{side}'] == 'HL') & (df_swings[f'sw_highs_{side}'].shift(1) == 'HH')),
                                   "UP",
                                   np.where(((df_swings[f'sw_lows_{side}'] == 'LL') & (df_swings[f'sw_highs_{side}'].shift(1) == 'LH')) |
                                            ((df_swings[f'sw_highs_{side}'] == 'LH') & (df_swings[f'sw_lows_{side}'].shift(1) == 'LL')),
                                            "DOWN",
                                            np.where(((df_swings[f'sw_lows_{side}'] == 'LL') & (df_swings[f'sw_highs_{side}'].shift(1) == 'HL')) |
                                                     ((df_swings[f'sw_lows_{side}'] == 'LL') & (df_swings[f'sw_highs_{side}'].shift(1) == 'HH')) |
                                                     ((df_swings[f'sw_lows_{side}'] == 'HL') & (df_swings[f'sw_highs_{side}'].shift(1) == 'LH')) |
                                                     ((df_swings[f'sw_highs_{side}'] == 'LH') & (df_swings[f'sw_lows_{side}'].shift(1) == 'HL')) |
                                                     ((df_swings[f'sw_highs_{side}'] == 'HH') & (df_swings[f'sw_lows_{side}'].shift(1) == 'LL')),
                                                     "UNCERTAIN",
                                                     np.nan
                                                    )
                                                 )
                                             )


    df_swings = df_swings[[f"sw_highs_{side}", f"sw_lows_{side}", f"trend_{side}"]]

    df_ffill = pd.concat([df, df_swings], axis=1)

    # Define the columns to be copied
    columns_to_copy = [f"sw_highs_{side}", f"sw_lows_{side}", f"trend_{side}"]

    # Forward fill columns in df_ffill
    df_ffill[columns_to_copy] = df_ffill[columns_to_copy].fillna(method='ffill')

    # Drop the columns to be replaced from df
    for column in columns_to_copy:
        if column in df.columns:
            df.drop(columns=column, inplace=True)

    # Update the original DataFrame with the filled columns
    df = pd.concat([df, df_ffill[columns_to_copy]], axis=1)

    df[f'trend_{side}'] = np.where(
                            ((df[f'sw_lows_{side}'] == 'HL') & (df['High'] > df[f'sw_high_price_{side}'])),
                               "UP",
                               np.where(((df[f'sw_highs_{side}'] == 'LH') & (df['Low'] < df[f'sw_low_price_{side}'])),
                                        "DOWN",
                                        df[f'trend_{side}']
                                       )
                            )
    # df['trend'].fillna(method='ffill', inplace=True)
    if side == "long":
        df[f'tsl_{side}'] = df[f'sw_low_price_{side}'].shift(1)
        df[f'{side}_entry'] = df[f'sw_high_price_{side}'].shift(1)
        
        df[f"{side}_Signal"] = np.where(
                            ((df[f'sw_lows_{side}'] == "HL") | ((df[f'sw_highs_{side}'] == "HH") & (df[f'sw_lows_{side}'] == "HL"))) & 
                            ((df['High'] > df[f'sw_high_price_{side}'].shift(1)) &
                            (df['High'].shift(1) < df[f'sw_high_price_{side}'].shift(1)) &
                            ((df[f'trend_{side}'].shift(1) == "UNCERTAIN") | (df[f'trend_{side}'] == "UP"))),
                            True,
                            False
                            )
        
        df[f"{side}_Exit"] = np.where((df[f'sw_highs_{side}'] == "LH") & 
                            (df[f'sw_trend_{side}'].shift(1) == 1.0) &
                            (df[f'sw_trend_{side}'] == -1.0) &
                            (df[f'trend_{side}'] == "UNCERTAIN"),  
                            ((df[f'sw_high_price_{side}'] - df[f'sw_low_price_{side}'])*exit_perc + df[f'sw_low_price_{side}']), 
                                np.nan)
        
        df["pi_top"] = np.where(
                            (df['Open'].rolling(window=111).mean()) > (df['Open'].rolling(window=350).mean() * 2), 
                            True, 
                            False
                            )
    
        
    else:
        df[f'tsl_{side}'] = df[f'sw_high_price_{side}'].shift(1)
        df[f'{side}_entry'] = df[f'sw_low_price_{side}'].shift(1)
        
        df[f"{side}_Signal"] = np.where(
                            ((df[f'sw_highs_{side}'] == "LH") | ((df[f'sw_lows_{side}'] == "LL") & (df[f'sw_highs_{side}'] == "LH"))) & 
                            (df['Low'] < df[f'sw_low_price_{side}'].shift(1)) &
                            (df['Low'].shift(1) > df[f'sw_low_price_{side}'].shift(1)) &
                            ((df[f'trend_{side}'].shift(1) == "UNCERTAIN") | (df[f'trend_{side}'] == "DOWN")),  
                            True, 
                            False
                            )

        df[f"{side}_Exit"] = np.where((df[f'sw_lows_{side}'] == "HL") & 
                            (df[f'sw_trend_{side}'].shift(1) == -1.0) &
                            (df[f'sw_trend_{side}'] == 1.0) &
                            (df[f'trend_{side}'] == "UNCERTAIN"),  
                            (df[f'sw_high_price_{side}'] - (df[f'sw_high_price_{side}'] - df[f'sw_low_price_{side}'])*exit_perc), 
                                np.nan)    

        df["pi_bottom"] = np.where(
                            (df['Close'].rolling(window=550).mean() > df['Close'].rolling(window=250).mean()), 
                            True, 
                            False
                            )
    
    st.write(df[300:500])
    
    return df

In [11]:
def backtest(df, ticker, direction="Both", commission=0.04/100, tp_perc_long=0, tp_perc_short=0, pi_exit=True, tsl_offset_long_en=True, tsl_offset_short_en=True, tsl_offset_long_pct=0.1/100, tsl_offset_short_pct=0.1/100, init_sl_offset_long=0.1/100, init_sl_offset_short=0.1/100):
    in_position = False
    buy_pos = False
    sell_pos = False

    results_df = pd.DataFrame()
    buydates, buyprices = [], []
    selldates, sellprices = [], []
    exit_types = []

    for index, row in df.iterrows():
    # ---------------------------------------------long position close check------------------------------
        if in_position and buy_pos:
            
            tsl = row.tsl_long*(1-tsl_offset_long)
            sl = max(init_sl, tsl)
            if (row.Low <= sl):
                selldates.append(index)
                sellprices.append(row.Low)
                in_position = False
                buy_pos = False
                exit_types.append("SL Hit")
            elif (row.High >= tp) and (tp_perc_long != 0):
                selldates.append(index)
                sellprices.append(tp)
                in_position = False
                buy_pos = False
                exit_types.append("TP Hit")
            elif (row.pi_top) and (pi_exit):
                selldates.append(index)
                sellprices.append(row.High)
                in_position = False
                buy_pos = False
                exit_types.append("Pi Cycle")
            elif row.long_Exit > 0:
                limit = row.long_Exit
            elif row.Low <= limit:
                selldates.append(index)
                sellprices.append(limit)
                in_position = False
                buy_pos = False
                exit_types.append("Limit price Hit due to uncertain trend")

    # ---------------------------------------------short position close check------------------------------
        elif in_position and sell_pos:
            tsl = row.tsl_short*(1+tsl_offset_short)
            sl = min(init_sl, tsl)
            if (row.High >= sl):
                buydates.append(index)
                buyprices.append(row.High)
                in_position = False
                sell_pos = False
                exit_types.append("SL Hit")
                
            elif (row.Low <= tp) and (tp_perc_short != 0):
                buydates.append(index)
                buyprices.append(tp)
                in_position = False
                buy_pos = False
                exit_types.append("TP Hit")
            elif (row.pi_bottom) and (pi_exit):
                buydates.append(index)
                buyprices.append(row.Low)
                in_position = False
                buy_pos = False
                exit_types.append("Pi Cycle")
            elif row.short_Exit > 0:
                limit = row.short_Exit
            elif row.High >= limit:
                buydates.append(index)
                buyprices.append(limit)
                in_position = False
                buy_pos = False
                exit_types.append("Limit price Hit due to uncertain trend")
                
    #         print(limit, in_position)
                
    # ======================================================================================================              
                
    # ---------------------------------------------long position entry check------------------------------
        if not in_position:
            if direction in ("Both", "Long") and row.long_Signal:
                buyprice = row.long_entry
                buydates.append(index)
                buyprices.append(buyprice)
                in_position = True
                buy_pos = True
                tp = buyprice * (1 + tp_perc_long)
                limit = np.nan
                init_sl = row.tsl_long*(1-init_sl_offset_long)
                tsl_offset_long = tsl_offset_long_pct if tsl_offset_long_en == True else 0
                
            elif direction in ("Both", "Short") and row.short_Signal:
                sellprice = row.short_entry
                selldates.append(index)
                sellprices.append(sellprice)
                in_position = True
                sell_pos = True
                tp = sellprice / (1 + tp_perc_short)
                limit = np.nan
                init_sl = row.tsl_short*(1+init_sl_offset_short)
                tsl_offset_short = tsl_offset_short_pct if tsl_offset_short_en == True else 0
                
    if len(buydates) == 0:
        print(f"No trades were made for {ticker}.")
    else:
        profits = [(sell - buy) / buy - commission for sell, buy in zip(sellprices, buyprices)]
        returns = ((pd.Series(profits, dtype=float) + 1).prod() - 1) * 100
        wins = 0
        for i in profits:
            if i > 0:
                wins += 1
            i += 1
        winrate = round((wins / len(buydates)) * 100, 2)
        ct = min(len(buydates), len(selldates))

        # BTCUSDT buy and hold returns during the same period
        buy_hold_ret = (df['Close'][-1] - df['Open'][0]) / df['Open'][0] * 100

        results_df = pd.concat([results_df, pd.DataFrame({'ticker': f'{ticker}', 'returns': [returns], 'winrate': [winrate], 'trades': [ct], 'buy&hold_ret%': [buy_hold_ret]})])
        st.subheader('Backtest Results')
        st.write(f'{ticker}, winrate={winrate}%, returns={round(returns, 2)}%, no. of trades = {ct}, buy&hold_ret = {round(buy_hold_ret, 2)}%')

    # Return the trade data along with other results
    return {
        'buydates': buydates,
        'buyprices': buyprices,
        'selldates': selldates,
        'sellprices': sellprices,
        'profits': profits,
        'Exit Type' : exit_types
        # Other results...
    }, results_df

def displayTrades(direction="Both", **kwargs):
    st.write('Direction: ', direction)
    # Access the trade data and other results from kwargs
    buydates = kwargs['buydates']
    buyprices = kwargs['buyprices']
    selldates = kwargs['selldates']
    sellprices = kwargs['sellprices']
    profits = kwargs['profits']
    exit = kwargs['Exit Type']

    ct = min(len(buydates), len(selldates))
    
    # Create a DataFrame to store the trades
    dfr = pd.DataFrame()
    dfr['buydates'] = buydates[:ct]
    dfr['buyprice'] = buyprices[:ct]
    dfr['selldates'] = selldates[:ct]
    dfr['sellprice'] = sellprices[:ct]
    dfr['profits'] = (profits[:ct])
    dfr['commulative_returns'] = ((pd.Series(profits) + 1).cumprod())
    dfr['Exit Type'] = exit[:ct]
    
    # Add a column to indicate the trade side
    dfr['tradeSide'] = np.where(dfr['buydates'] < dfr['selldates'], 'Long', 'Short')
    
    return dfr

# Defining the variables

In [12]:
ticker = "BTCUSDT"
timeframe = "1d"
start = "2019-09-08"
start_date = datetime.strptime(start, "%Y-%m-%d")
end = str(datetime.now())
# Calculate the difference in days
day = (datetime.now() - start_date).days
direction = 'Both' # other options for direction 'Long' or 'Short'

In [13]:
max_sw_cnt_l = 3
max_sw_cnt_s = 3

In [14]:
df = getdata(ticker, timeframe, day)

In [15]:
init_sl_offset_long = 0.1 * 0.01     #----> streamlit input
init_sl_offset_short = 0.1 * 0.01    #----> streamlit input
tp_exit_long = False  #----> streamlit input
tp_exit_short = False  #----> streamlit input
tp_value_long = 15 * 0.01 #----> streamlit input
tp_value_short = 38 * 0.01 #----> streamlit input
tp_perc_long = 0 if tp_exit_long == False else tp_value_long
tp_perc_short = 0 if tp_exit_short == False else tp_value_short

exit_limit_long_en = True   #----> streamlit input
exit_limit_short_en = True  #----> streamlit input
exit_perc_value = 80 * 0.01 #----> streamlit input
exit_perc_long = 0 if exit_limit_long_en == False else exit_perc_value
exit_perc_short = 0 if exit_limit_short_en == False else exit_perc_value

direction = "Both"  #----> streamlit input
pi_exit = True  #----> streamlit input

tsl_offset_pct = 0.1
tsl_offset_long_en = True   #----> streamlit input
tsl_offset_long = tsl_offset_pct*0.01 if tsl_offset_long_en == True else 0 #----> streamlit input
tsl_offset_short_en = False  #----> streamlit input
tsl_offset_short = tsl_offset_pct*0.01 if tsl_offset_short_en == True else 0 #----> streamlit input



# no use found in the tradingview script
# entry_offset_en = False
# entry_offset_pct = 0.1
# entry_offset = entry_offset_pct/100 if entry_offset_en == True else 0

In [16]:
dfl = calculate_gann_signals(df, max_sw_cnt = max_sw_cnt_l, exit_perc = exit_perc_long, side = "long")
dfs = calculate_gann_signals(df, max_sw_cnt = max_sw_cnt_s, exit_perc = exit_perc_short, side = "short")
unique_columns = dfs.columns.difference(dfl.columns)
df = pd.concat([dfl, dfs[unique_columns]], axis=1)

2023-10-14 12:50:21.148 
  command:

    streamlit run c:\Users\abbAxi\kraken\k_env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [17]:
df

,Open,High,Low,Close,Volume,candle_type,sw_cnt_long,sw_trend_long,trend_cnt_long,sw_top_long,sw_bottom_long,sw_high_price_long,sw_low_price_long,sw_highs_long,sw_lows_long,trend_long,tsl_long,long_entry,long_Signal,long_Exit,pi_top,pi_bottom,short_Exit,short_Signal,short_entry,sw_bottom_short,sw_cnt_short,sw_high_price_short,sw_highs_short,sw_low_price_short,sw_lows_short,sw_top_short,sw_trend_short,trend_cnt_short,trend_short,tsl_short
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-09-08,10000.00,10412.65,10000.00,10391.63,3096.291,,0,NaN,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,NaN,False,NaN,False,0,NaN,NaN,NaN,NaN,False,NaN,0,NaN,NaN
2019-09-09,10316.62,10475.54,10077.22,10307.00,14824.373,up_bar,1,NaN,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,NaN,False,NaN,False,1,NaN,NaN,NaN,NaN,False,NaN,0,NaN,NaN
2019-09-10,10307.00,10382.97,9940.87,10102.02,9068.955,down_bar,-1,NaN,1,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,NaN,False,NaN,False,-1,NaN,NaN,NaN,NaN,False,NaN,1,NaN,NaN
2019-09-11,10094.27,10293.11,9884.31,10159.55,10897.922,down_bar,-2,NaN,1,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,NaN,False,NaN,False,-2,NaN,NaN,NaN,NaN,False,NaN,1,NaN,NaN
2019-09-12,10163.06,10450.13,10042.12,10415.13,15609.634,up_bar,1,NaN,1,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,NaN,False,NaN,False,1,NaN,NaN,NaN,NaN,False,NaN,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-10,27578.40,27727.30,27281.00,27381.20,239779.496,,-2,1.0,9,False,False,28613.0,26050.0,HH,HL,UP,26050.0,28613.0,False,NaN,False,False,NaN,False,26050.0,False,-2,28613.0,HH,26050.0,HL,False,1.0,9,UP,28613.0
2023-10-11,27381.30,27467.60,26525.00,26864.50,388186.571,down_bar,-3,-1.0,10,False,False,28613.0,26050.0,HH,HL,UP,26050.0,28613.0,False,NaN,False,False,NaN,False,26050.0,False,-3,28613.0,HH,26050.0,HL,False,-1.0,10,UP,28613.0
2023-10-12,26864.50,26939.00,26534.80,26744.00,220279.040,,-3,-1.0,1,False,False,28613.0,26050.0,HH,HL,UP,26050.0,28613.0,False,NaN,False,False,NaN,False,26050.0,False,-3,28613.0,HH,26050.0,HL,False,-1.0,1,UP,28613.0


In [18]:
# Define the side ('long' or 'short') to plot chart
showSide = 'long'

# Backtest

In [23]:
results_data, result_df = backtest(df, ticker, direction="Both", commission=0.04/100, tp_perc_long=tp_perc_long, tp_perc_short=tp_perc_short, pi_exit = pi_exit, tsl_offset_long_en=tsl_offset_long_en, tsl_offset_short_en=tsl_offset_short_en, tsl_offset_long_pct=tsl_offset_long, tsl_offset_short_pct=tsl_offset_short, init_sl_offset_long=init_sl_offset_long, init_sl_offset_short=init_sl_offset_short)

In [24]:
result_df

,ticker,returns,winrate,trades,buy&hold_ret%
0,BTCUSDT,814.631673,56.52,22,168.532


In [25]:
dfr = displayTrades(direction="Both", **results_data)
dfr

,buydates,buyprice,selldates,sellprice,profits,commulative_returns,Exit Type,tradeSide
0,2019-10-25,8800.00,2019-10-23,7700.67,-0.125324,0.874676,SL Hit,Short
1,2020-01-06,7691.00,2020-03-08,7997.70,0.039478,0.909206,SL Hit,Long
2,2020-04-02,7290.00,2020-03-08,8409.00,0.153098,1.048404,SL Hit,Short
3,2020-04-02,6960.71,2020-07-13,9339.28,0.341314,1.406239,Limit price Hit due to uncertain trend,Long
4,2020-07-22,9470.00,2020-09-03,9901.16,0.045129,1.469701,SL Hit,Long
5,2020-10-10,11494.00,2020-09-03,11130.00,-0.032069,1.422570,SL Hit,Short
6,2020-10-10,11177.00,2021-04-13,63850.00,4.712224,8.126036,Pi Cycle,Long
7,2021-04-13,61950.00,2021-04-14,64986.11,0.048609,8.521035,Pi Cycle,Long
8,2021-06-14,41000.00,2021-05-13,46930.43,0.144245,9.750148,SL Hit,Short
9,2021-06-14,39470.00,2021-06-22,28780.01,-0.271238,7.105534,SL Hit,Long


## CANDLES CHART

In [22]:
plot_advanced_gann_swing_chart(df, dfr, visible_data_points=300, side=showSide)

In [ ]:
styled_df = df.copy()
styled_df.reset_index(inplace = True)
# Define custom CSS to make the header row sticky
css = [
    {
        'selector': 'thead tr',
        'props': 'position: sticky; top: 0; background-color: white; color: black;'
    }
]

# Apply the custom CSS to the DataFrame
styled_df = styled_df.style.set_table_styles(css)

# Display the styled DataFrame
styled_df

# Set Final Bot Parameters

In [ ]:
# please put this bot parameters here

ticker = "BTCUSDT"
timeframe = "15m"
usdt_amount = 15

takeLong = True 
takeShort = True

mult = 1 # TP R:R MULTIPLYER                           ----> TO BE OPTIMIZED
risk = 15.0 # IF STOPLOSS > ?%  DONT ENTER!            ----> TO BE OPTIMIZED
HL = 20 # rolling high lows                            ----> TO BE OPTIMIZED

In [ ]:
# Trades Display

# def displayTrades():
#     ct = min(len(buydates),len(selldates))
#     dfr =pd.DataFrame()
#     dfr['buydates']= buydates[:ct]
#     dfr['buyprice']= buyprices[:ct]
#     dfr['selldates'] = selldates[:ct]
#     dfr['sellprice'] = sellprices[:ct]
#     dfr['profits'] = (profits[:ct])
#     dfr['commulative_returns'] = ((pd.Series(profits) + 1).cumprod())
#     dfr['tradeSide'] = np.where(dfr['buydates'] < dfr['selldates'], 'Long', 'Short')
#     return dfr

